# IPython interface for running GPU DSM + fitting on cluster

In [32]:
from dsm_tools import ssh_init, job_start, job_done, job_transfer_clean, uuid, os, server, username, password, Calculation, CalculationStar, CalculationLinear
from fdt_fit import fdt_fit, fdt, fdtvec
import numpy as np
import matplotlib.pyplot as plt

In [115]:
class iterationChain(object):
    w = None #Weight fraction of linear chains
    chain = []

    def add(self, calcs):
        self.chain.append(calcs)
    def __init__(self, w):
        self.w = w

In [187]:
def mix_spectra(w, x, y):
    xWeight = w #Fraction of linear
    yWeight = 1 - w

    #Calculate p_eq of the mix
    mix_lambdaArr = np.append(x.lambdaArr, y.lambdaArr)
    mix_gArr = np.append(np.multiply(x.gArr, x.lambdaArr)/np.dot(x.gArr, x.lambdaArr) * xWeight, np.multiply(y.gArr, y.lambdaArr)/np.dot(y.gArr, y.lambdaArr) * yWeight)

    #Sort by relaxation times tau
    sort = np.argsort(mix_lambdaArr)
    mix_lambdaArr = mix_lambdaArr[sort]
    mix_gArr = mix_gArr[sort]

    return mix_lambdaArr, np.divide(mix_gArr,mix_lambdaArr)/np.sum(np.divide(mix_gArr,mix_lambdaArr))

In [34]:
nChains = 5000
CDtoggle = 0
simTime = 1000000
num_gpu = 4

#Linear chain
nk = 50

#Star chain
nArms = 3
nkArms = [6, 6, 6]

To start a claculation of a **linear chain**, call `CalculationLinear(Nc, CDtoggle, Nchains, Ngpus, simTime)`

To start a claculation of a **star-branched chain**, call `CalculationStar(nArms, NcArms, CDtoggle, Nchains, Ngpus, simTime)`

### Zeroth iteration (no constraint dynamics)

In [35]:
x, y = CalculationLinear(nk, 0, nChains, num_gpu, simTime), CalculationStar(nArms, nkArms, 0, nChains, num_gpu, simTime)

In [37]:
x.calc()
y.calc()

In [197]:
x.check_self()
y.check_self()

In [54]:
x.plot_fit_results()

In [47]:
y.plot_fit_results()

### Longest relaxation time for a pure linear and a pure star

In [120]:
weights = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0]

In [126]:
iterations = [iterationChain(w) for w in weights]

In [134]:
iterations[1].add((CalculationLinear(nk, 1, nChains, num_gpu, simTime), CalculationStar(nArms, nkArms, 1, nChains, num_gpu, simTime)))

In [139]:
iterations[1].chain[-1]

In [193]:
a,b = mix_spectra(0.9, x, y)
iterations[1].chain[-1][0].set_pcd_cr_input(a,b)
iterations[1].chain[-1][1].set_pcd_cr_input(a,b)

In [198]:
iterations[1].chain[-1][0].calc()
iterations[1].chain[-1][1].calc()

In [204]:
iterations[1].chain[-1][0].check_self()
iterations[1].chain[-1][1].check_self()

In [205]:
iterations[1].chain[-1][0].plot_fit_results()

In [207]:
iterations[1].chain[-1][0].fdt_y

In [210]:
fdt_result=zip(iterations[1].chain[-1][0].fdt_x, iterations[1].chain[-1][0].fdt_y)
file = open("fdt_result.dat","w")
for i in fdt_result:
    file.write(str(i[0])+'\t'+str(i[1])+'\n')
file.close()